In [1]:
# Import dependcies

import pandas as pd
import numpy as np
import psycopg2
from config import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler 
import tensorflow as tf

In [2]:
# Connect to database and extract mldata table
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port= PORT)
cursor = conn.cursor()
cursor.execute("SELECT * FROM mldata")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
mldata = pd.DataFrame(data, columns=colnames)
print(mldata.shape)
mldata.head()

(7940, 22)


,index,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,...,weather_dry,weather_wet,weather_cloudy,Win,podium,age,driverposition,driverwin,constructorposition,constructorwin
0,0,2008,1,1,18,1,1,1,1,1,...,0,0,0,1,1,23.0,0.0,0.0,0.0,0.0
1,1,2008,1,2,18,1,2,2,5,2,...,0,0,0,0,2,31.0,0.0,0.0,0.0,0.0
2,2,2008,1,3,18,1,3,3,7,3,...,0,0,0,0,3,23.0,0.0,0.0,0.0,0.0
3,3,2008,1,4,18,1,4,4,11,4,...,0,0,0,0,0,27.0,0.0,0.0,0.0,0.0
4,4,2008,1,5,18,1,5,1,3,5,...,0,0,0,0,0,26.0,0.0,0.0,0.0,0.0


In [3]:
mldata.columns

Index(['index', 'year', 'round', 'resultId', 'raceId', 'circuitId', 'driverId',
       'constructorId', 'grid', 'position', 'weather_warm', 'weather_cold',
       'weather_dry', 'weather_wet', 'weather_cloudy', 'Win', 'podium', 'age',
       'driverposition', 'driverwin', 'constructorposition', 'constructorwin'],
      dtype='object')

In [4]:
# Replace all null values with 0s
mldata = mldata.replace(np.nan, 0)

In [5]:
# Convert float64 columns to integers
mldata[['age', 'driverposition', 'driverwin', 
       'constructorposition', 'constructorwin']] = mldata[['age', 'driverposition', 'driverwin', 
                                                         'constructorposition', 'constructorwin']].astype(int)

## One Hot Encode categorical variables and merge with mldata

In [6]:
# One hot encode circuitId

enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
df = pd.DataFrame(enc.fit_transform(mldata.circuitId.values.reshape(-1,1)))

# Rename encoded columns
df.columns = enc.get_feature_names(['circuitId'])
print(df.shape)

(7940, 32)


In [7]:
# One hot encode driverId

enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
df1 = pd.DataFrame(enc.fit_transform(mldata.driverId.values.reshape(-1,1)))

# Rename encoded columns
df1.columns = enc.get_feature_names(['driverId'])
print(df1.shape)

(7940, 112)


In [8]:
# One hot encode constructorId

enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
df2 = pd.DataFrame(enc.fit_transform(mldata.constructorId.values.reshape(-1,1)))

# Rename encoded columns
df2.columns = enc.get_feature_names(['constructorId'])
print(df2.shape)

(7940, 34)


In [9]:
# Merge mldata with df
mldata = mldata.merge(df,left_index=True,right_index=True)
print(mldata.shape)

(7940, 54)


In [10]:
# Now Merge mldata with df1
mldata = mldata.merge(df1,left_index=True,right_index=True)
print(mldata.shape)

(7940, 166)


In [11]:
# Finally Merge mldata with df2
mldata = mldata.merge(df2,left_index=True,right_index=True)
print(mldata.shape)

(7940, 200)


## Building the Race Winner Predictor

In [12]:
# Make a copy of mldata in order to create test and train datasets
nn = mldata.copy()

In [13]:
# Train the data with all outcomes before 2017

# We are dropping circuitId, driverId and constructorId because we have included the dummified columns
    # regarding those variables. 

train = nn[nn['year']<=2016]
X_train = train.drop(['resultId', 'raceId', 'index', 'circuitId', 'driverId','constructorId', 'position', 'Win', 'podium', 'age',
                      'driverposition', 'driverwin', 'constructorposition', 'constructorwin'], axis=1)
y_train = train.Win

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

# Create the test dataset
test = nn[nn.year == 2017]
##test = test.reset_index()
##test = test.drop(["index"], axis=1)

X_test = test.drop(['resultId', 'raceId', 'index', 'circuitId', 'driverId','constructorId', 'position', 'Win', 'podium', 'age',
                      'driverposition', 'driverwin', 'constructorposition', 'constructorwin'], axis=1)
y_test = test.Win
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [14]:
X_test

,year,round,grid,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,circuitId_1,circuitId_2,...,constructorId_131,constructorId_164,constructorId_166,constructorId_205,constructorId_206,constructorId_207,constructorId_208,constructorId_209,constructorId_210,constructorId_211
0,1.767531,-1.620866,-1.468562,-0.754622,-0.057397,-0.474433,-0.371099,-0.353435,4.148074,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
1,1.767531,-1.620866,-1.626589,-0.754622,-0.057397,-0.474433,-0.371099,-0.353435,4.148074,-0.241768,...,4.861312,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
2,1.767531,-1.620866,-1.310535,-0.754622,-0.057397,-0.474433,-0.371099,-0.353435,4.148074,-0.241768,...,4.861312,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
3,1.767531,-1.620866,-1.152509,-0.754622,-0.057397,-0.474433,-0.371099,-0.353435,4.148074,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
4,1.767531,-1.620866,-0.994482,-0.754622,-0.057397,-0.474433,-0.371099,-0.353435,4.148074,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1.767531,1.938374,0.901836,1.325167,-0.057397,-0.474433,-0.371099,-0.353435,-0.241076,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
396,1.767531,1.938374,1.217889,1.325167,-0.057397,-0.474433,-0.371099,-0.353435,-0.241076,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
397,1.767531,1.938374,0.585783,1.325167,-0.057397,-0.474433,-0.371099,-0.353435,-0.241076,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0
398,1.767531,1.938374,0.111704,1.325167,-0.057397,-0.474433,-0.371099,-0.353435,-0.241076,-0.241768,...,-0.205706,-0.132735,-0.107114,-0.107114,-0.128599,-0.130386,-0.153381,-0.108531,-0.079424,0.0


In [15]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=150, activation="tanh", input_dim=186))
nn_model.add(tf.keras.layers.Dense(units=100, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=50, activation="relu"))
# nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))
# nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 150)               28050     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 48,721
Trainable params: 48,721
Non-trainable params: 0
_________________________________________________________________


In [16]:
nn_model.compile(optimizer="SGD", loss="binary_crossentropy", metrics=['accuracy'])

In [17]:
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50
210/210 [==============================] - 1s 1ms/step - loss: 0.4016 - accuracy: 0.8607
Epoch 2/50
210/210 [==============================] - 0s 1ms/step - loss: 0.2073 - accuracy: 0.9537
Epoch 3/50
210/210 [==============================] - 0s 1ms/step - loss: 0.1873 - accuracy: 0.9537
Epoch 4/50
210/210 [==============================] - 0s 1ms/step - loss: 0.1809 - accuracy: 0.9537
Epoch 5/50
210/210 [==============================] - 0s 984us/step - loss: 0.1772 - accuracy: 0.9537
Epoch 6/50
210/210 [==============================] - 0s 1ms/step - loss: 0.1743 - accuracy: 0.9537
Epoch 7/50
210/210 [==============================] - 0s 1ms/step - loss: 0.1717 - accuracy: 0.9537
Epoch 8/50
210/210 [==============================] - 0s 978us/step - loss: 0.1692 - accuracy: 0.9537
Epoch 9/50
210/210 [==============================] - 0s 976us/step - loss: 0.1666 - accuracy: 0.9537
Epoch 10/50
210/210 [==============================] - 0s 939us/step - loss: 0.1639 - accuracy

In [18]:
nn_model.evaluate(X_test, y_test,verbose=2)

13/13 - 0s - loss: 0.1284 - accuracy: 0.9500


[0.12835340201854706, 0.949999988079071]

In [19]:
predictions = nn_model.predict(X_test)
# Create dataframe with prediction probabilities
pred = pd.DataFrame(predictions, columns=['proba_1'])
pred.head()

,proba_1
0,0.191968
1,0.376089
2,0.011675
3,0.042390
4,0.029140


In [20]:
Z1 = test[['resultId', 'raceId', 'round', 'circuitId', 'driverId', 'position', 'Win', 'podium']].copy()
Z1 = Z1.reset_index()
Z1 = Z1.drop(["index"], axis=1)

# Merge Z1 and pred
Z_final = Z1.merge(pred,left_index=True,right_index=True)
Z_final.head()

,resultId,raceId,round,circuitId,driverId,position,Win,podium,proba_1
0,23379,969,1,1,20,1,1,1,0.191968
1,23380,969,1,1,1,2,0,2,0.376089
2,23381,969,1,1,822,3,0,3,0.011675
3,23382,969,1,1,8,4,0,0,0.042390
4,23383,969,1,1,830,5,0,0,0.029140


In [21]:
# We will groupby 'round' and take the max probability for a driver to win
maxprob = Z_final.groupby(['round']).agg({'proba_1':'max'})
maxprob = maxprob.reset_index()
maxprob = maxprob.rename(columns={'proba_1':'proba_1_max'})

# We will then merge the proba_1_max column to Z_Final
Z_final = pd.merge(Z_final, maxprob, how='left', on=['round'])
print(Z_final.shape)
Z_final.head()

(400, 10)


,resultId,raceId,round,circuitId,driverId,position,Win,podium,proba_1,proba_1_max
0,23379,969,1,1,20,1,1,1,0.191968,0.376089
1,23380,969,1,1,1,2,0,2,0.376089,0.376089
2,23381,969,1,1,822,3,0,3,0.011675,0.376089
3,23382,969,1,1,8,4,0,0,0.042390,0.376089
4,23383,969,1,1,830,5,0,0,0.029140,0.376089


## Predicting the Race Winner

In [22]:
# Number of times our predicted winner is the actual winner
winner = []
for index, row in Z_final.iterrows():
    if row['proba_1'] == row['proba_1_max']:
        if row['podium'] in [1]:
            winner.append(1)
        else:
            winner.append(0)
    else:
        winner.append(0)
        
Z_final['winner'] = winner
Z_final['winner'].sum()

10

In [23]:
# Number of times our predicted winner is in the top 2
top2 = []
for index, row in Z_final.iterrows():
    if row['proba_1'] == row['proba_1_max']:
        if row['podium'] in [1,2]:
            top2.append(1)
        else:
            top2.append(0)
    else:
        top2.append(0)
        
Z_final['top2'] = top2
Z_final['top2'].sum()

14

In [24]:
# Number of times our predicted winner is in the top 2
top3 = []
for index, row in Z_final.iterrows():
    if row['proba_1'] == row['proba_1_max']:
        if row['podium'] in [1,2,3]:
            top3.append(1)
        else:
            top3.append(0)
    else:
        top3.append(0)
        
Z_final['top3'] = top3
Z_final['top3'].sum()

14

## Processing Z_final for predicting all 20 Race outcomes

In [25]:
# Make a copy of svm_pred
X = Z_final.copy()
# Sort X in ascending order
X = X.drop(['proba_1_max', 'Win', 'podium', 'winner', 'top2', 'top3'], axis=1)
X = X.sort_values(["round", "proba_1"], ascending = (True, False))
X.head()

,resultId,raceId,round,circuitId,driverId,position,proba_1
1,23380,969,1,1,1,2,0.376089
0,23379,969,1,1,20,1,0.191968
16,23395,969,1,1,817,0,0.138148
3,23382,969,1,1,8,4,0.042390
4,23383,969,1,1,830,5,0.029140


In [26]:
# Create a forLoop and add a counter to each row. Once the raceId changes, the counter will refresh
# This counter will represent the driver's final predicted race outcome

round = [1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
count = 1
prediction = []

for i in round:
    count = 0
    for index, row in X.iterrows():
        if row["round"] == i:
            count = count +1
            prediction.append(count)
        else:
            continue
        

X["Pred_Position"] = prediction
X.head()

,resultId,raceId,round,circuitId,driverId,position,proba_1,Pred_Position
1,23380,969,1,1,1,2,0.376089,1
0,23379,969,1,1,20,1,0.191968,2
16,23395,969,1,1,817,0,0.138148,3
3,23382,969,1,1,8,4,0.042390,4
4,23383,969,1,1,830,5,0.029140,5


In [27]:
# We will create a new column which shows the delta between actual and predicted finishing position 
X['Delta'] = X['position'] - X['Pred_Position']
X['Delta'] = X['Delta'].abs()

In [28]:
# If the delta is within 2, we will assign the number 1 to it. Otherwise, we will assign 0. 
X["Spread"] = X["Delta"].copy()
X["Spread"] = X["Spread"].apply(lambda x: 1 if x<= 2 else 0)

In addition to predicting every race position (with a spread of 2), we also want to predict the driver's finishing group or bin. To do that, we will create additional columns where we will map the predicted and actual finishing positions to their respective bins: Podium, topsix, topten and else. 

In [29]:
# Add two more columns to create ending position bins

X["pos_group"] = X["position"].copy()
X["pred_pos_group"] = X["Pred_Position"].copy()

podium = [1,2,3]
topsix = [4,5,6]
topten = [7,8,9,10]

def position_group(x):
    if x in podium:
        return 1
    if x in topsix:
        return 2
    if x in topten:
        return 3
    else:
        return 4
    

X["pos_group"] = X["pos_group"].apply(position_group)  
X["pred_pos_group"] = X["pred_pos_group"].apply(position_group)  

In [30]:
# This is the final dataframe which we will use to calute the prediction percentages. 
print(X.shape)
X

(400, 12)


,resultId,raceId,round,circuitId,driverId,position,proba_1,Pred_Position,Delta,Spread,pos_group,pred_pos_group
1,23380,969,1,1,1,2,0.376089,1,1,1,1,1
0,23379,969,1,1,20,1,0.191968,2,1,1,1,1
16,23395,969,1,1,817,0,0.138148,3,3,0,4,1
3,23382,969,1,1,8,4,0.042390,4,0,1,2,2
4,23383,969,1,1,830,5,0.029140,5,0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
389,23771,988,20,24,13,10,0.002233,16,6,0,3,4
397,23779,988,20,24,840,18,0.002045,17,1,1,4,4
395,23777,988,20,24,842,16,0.002007,18,2,1,4,4
396,23778,988,20,24,828,17,0.001940,19,2,1,4,4


## Predicting all 20 race positions with a spread of 2

In [31]:
correct = len(X.loc[X["Spread"]==1])
total = len(X)
score = (correct / total) * 100

print(f"Model predicted {score} percentage of all outcomes within a spread of 2 positions")

Model predicted 36.75 percentage of all outcomes within a spread of 2 positions


## Predicting driver finishing bins

In [32]:
group_correct = len(X.loc[X["pos_group"]==X["pred_pos_group"]])
total = len(X)
group_score = (group_correct / total) * 100
print(f"Model predicted {group_score} percentage of all outcome groups for 2017")

Model predicted 52.75 percentage of all outcome groups for 2017


In [33]:
podium_percentage = len(X.loc[(X["pos_group"]==1) & (X["pos_group"]==X["pred_pos_group"])]) / len(X.loc[X["pos_group"]==1])
topsix_percentage = len(X.loc[(X["pos_group"]==2) & (X["pos_group"]==X["pred_pos_group"])]) / len(X.loc[X["pos_group"]==2])
topten_percentage = len(X.loc[(X["pos_group"]==3) & (X["pos_group"]==X["pred_pos_group"])]) / len(X.loc[X["pos_group"]==3])
bottomten_percentage = len(X.loc[(X["pos_group"]==4) & (X["pos_group"]==X["pred_pos_group"])]) / len(X.loc[X["pos_group"]==4])

print(f"Model predicted {podium_percentage * 100} percentage of all podiums for 2017")
print(f"Model predicted {topsix_percentage * 100} percentage of all topsix for 2017")
print(f"Model predicted {topten_percentage * 100} percentage of all topten for 2017")
print(f"Model predicted {bottomten_percentage * 100} percentage of all topten for 2017")

Model predicted 61.66666666666667 percentage of all podiums for 2017
Model predicted 31.666666666666664 percentage of all topsix for 2017
Model predicted 26.25 percentage of all topten for 2017
Model predicted 67.0 percentage of all topten for 2017


## Create Final Output File

In [34]:
nn_output = X[['resultId', 'Pred_Position', 'pred_pos_group']]
nn_output = nn_output.rename(columns={"Pred_Position": "NN_Outcome", "pred_pos_group": "NN_Bin"})

In [35]:
# Extract CSV file
nn_output.to_csv('../../../Resources/PythonExport/ML_Export/2017/nn_output_2017.csv')